  # This notebook splits the data into training, validation and test sets. Also pre-processes the data
  Data is split by instance, for each problem set 6 instances are used for training, 2 for validation, and 1 for testing

In [21]:
import pandas as pd

In [50]:
column_names = [
    'iterations',
    'instance_type',
    'tw_spread',
    'operator_selection_mechanism',
    'number_of_vertices_to_remove',
    "delta_last_improv", #change to rel_delta... for future work when i want to test on my benchmarks
    "acceptance_ratio",
    "i_last_improv",
    'prev_remove_operator',
    'prev_insert_operator',
    'route_imbalance',
    'capacity_utilization',
    'success_r_op_1',
    'success_r_op_2',
    'success_r_op_3',
    'success_r_op_4',
    'success_r_op_5',
    'success_i_op_1',
    'success_i_op_2',
    'success_i_op_3',
    'delta_cost',
    'new_cost',
    'chosen_remove_operator',
    'chosen_insert_operator'
]


log_df = pd.read_csv("../training_alns_iterations.log", sep=',', header=None, names=column_names)

In [51]:
log_df['iterations'] = log_df['iterations'].apply(lambda x: int(str(x).split(":")[-1]))

In [52]:
log_df['current_cost'] = log_df['new_cost'] - log_df['delta_cost']

In [53]:
#relativise delta last improv:
log_df['rel_delta_last_improv'] = log_df['delta_last_improv'] / log_df['current_cost']

In [54]:
log_df[['delta_last_improv', 'current_cost', 'rel_delta_last_improv']].head()

,delta_last_improv,current_cost,rel_delta_last_improv
0,0.000000,10267.032510,0.000000
1,-718.948672,9548.083839,-0.075298
2,-91.810910,9456.272928,-0.009709
3,-160.354919,9295.918009,-0.017250
4,-160.354919,9295.918009,-0.017250


In [55]:
def compute_short_long_term_improvement(df, alpha=0.5, k=20):
    improvements = []

    for i in range(len(df)):
        current_cost = df.loc[i, 'current_cost']
        immediate = df.loc[i, 'delta_cost']
        future_window = df['delta_cost'].iloc[i+1:i+1+k]

        # Best improvement (most negative value) in the next k steps
        best_future_improvement = future_window.min() if not future_window.empty else 0

        # Weighted combination
        rel_immediate = immediate / current_cost #added
        rel_best_future_improvement = best_future_improvement / current_cost #added
        combined = alpha * rel_immediate + (1 - alpha) * rel_best_future_improvement
        improvements.append(combined)

    df['short_long_improvement'] = improvements
    return df

In [56]:
df = compute_short_long_term_improvement(log_df, alpha=0.7, k=20)

In [65]:
df.head()

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_r_op_4,success_r_op_5,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,-718.948672,9548.083839,1,2,-557.001712
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,-91.810910,9456.272928,1,3,-118.005279
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,1.0,1.0,-160.354919,9295.918009,4,1,-165.986085
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.000000,9295.918009,5,2,-53.737642
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,1.0,1.0,0.000000,9295.918009,3,1,-53.737642


In [57]:
# 2nd with relative short_long_term_improvement
df.head()

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,current_cost,rel_delta_last_improv,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,-718.948672,9548.083839,1,2,10267.032510,0.000000,-0.054251
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,-91.810910,9456.272928,1,3,9548.083839,-0.075298,-0.012359
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,-160.354919,9295.918009,4,1,9456.272928,-0.009709,-0.017553
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,0.000000,9295.918009,5,2,9295.918009,-0.017250,-0.005781
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,0.000000,9295.918009,3,1,9295.918009,-0.017250,-0.005781


In [6]:
df.count()
# all columns are complete except for pre_remove_operator and prev_insert_operator since the first iteration in every alns run has no previous values

iterations                      216000
instance_type                   216000
tw_spread                       216000
operator_selection_mechanism    216000
number_of_vertices_to_remove    216000
delta_last_improv               216000
acceptance_ratio                216000
i_last_improv                   216000
prev_remove_operator            215892
prev_insert_operator            215892
route_imbalance                 216000
capacity_utilization            216000
success_r_op_1                  216000
success_r_op_2                  216000
success_r_op_3                  216000
success_r_op_4                  216000
success_r_op_5                  216000
success_i_op_1                  216000
success_i_op_2                  216000
success_i_op_3                  216000
delta_cost                      216000
new_cost                        216000
chosen_remove_operator          216000
chosen_insert_operator          216000
short_long_improvement          216000
dtype: int64

## Splitting in train, validation and test sets

In [58]:
problem_types = 6
problem_instances = 9
runs_per_instance = 2
iterations_per_run = 2000

In [59]:
# 1. divide dataframe by problem types
training_set, validation_set, test_set = [], [], []
problem_size = problem_instances * runs_per_instance * iterations_per_run
for start in range(0, df.shape[0], problem_size):
    print(" / problem type start: ", start)
    df_by_problem_type = df.iloc[start:start + problem_size]
    # divide in two, first part was done using roulette-wheel, second part was done using random selector
    df_roulette_wheel = df_by_problem_type.iloc[0:18000]
    df_random = df_by_problem_type.iloc[18000:]
    # from each problem type take three instances - two for validation and one for testing
    # training sets:
    df_roulette_wheel_train = df_roulette_wheel.iloc[0:2000*6]
    df_random_train = df_random.iloc[0:2000*6]
    # validation sets:
    df_roulette_wheel_val = df_roulette_wheel.iloc[2000*6:2000*8]
    df_random_val = df_random.iloc[2000*6:2000*8]
    # testing sets:
    df_roulette_wheel_test = df_roulette_wheel.iloc[2000*8:]
    df_random_test = df_random.iloc[2000*8:]

    print("train length of both operator selectors: ", df_roulette_wheel_train.shape[0], df_random_train.shape[0])
    print("validation length of both operator selectors: ", df_roulette_wheel_val.shape[0], df_random_val.shape[0])
    print("test length of both operator selectors: ", df_roulette_wheel_test.shape[0], df_random_test.shape[0])

    training_subset = pd.concat([df_roulette_wheel_train, df_random_train])
    training_set.append(training_subset)

    validation_subset = pd.concat([df_roulette_wheel_val, df_random_val])
    validation_set.append(validation_subset)

    test_subset = pd.concat([df_roulette_wheel_test, df_random_test])
    test_set.append(test_subset)

    # process_data(df_subset)

 / problem type start:  0
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  36000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  72000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  108000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  144000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  180000
train lengt

In [60]:
training_set   = pd.concat(training_set, ignore_index=True)
validation_set = pd.concat(validation_set,   ignore_index=True)
test_set       = pd.concat(test_set,  ignore_index=True)

In [61]:
print(training_set.shape)
print(validation_set.shape)
print(test_set.shape)

(144000, 27)
(48000, 27)
(24000, 27)


In [62]:
training_set.to_csv('training_set.csv', header=True, index=False)
validation_set.to_csv('validation_set.csv', header=True, index=False)
test_set.to_csv('test_set.csv', header=True, index=False)

## Prepare scalers for the data

In [42]:
import joblib
from sklearn.compose      import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

In [63]:
train_df = pd.read_csv("training_set.csv")

In [64]:
train_df.head()


,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,current_cost,rel_delta_last_improv,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,-718.948672,9548.083839,1,2,10267.032510,0.000000,-0.054251
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,-91.810910,9456.272928,1,3,9548.083839,-0.075298,-0.012359
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,-160.354919,9295.918009,4,1,9456.272928,-0.009709,-0.017553
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,0.000000,9295.918009,5,2,9295.918009,-0.017250,-0.005781
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,0.000000,9295.918009,3,1,9295.918009,-0.017250,-0.005781


In [66]:
# Prepare scaler for input features
num_pos    = ["iterations", "acceptance_ratio", "number_of_vertices_to_remove", "i_last_improv", "route_imbalance", "capacity_utilization", 'success_r_op_1', 'success_r_op_2',                     'success_r_op_3', 'success_r_op_4', 'success_r_op_5', 'success_i_op_1', 'success_i_op_2','success_i_op_3']      # always positive

num_signed = ["rel_delta_last_improv"]           # can be positive or negative

cat_cols   = ["instance_type", "tw_spread", "operator_selection_mechanism",
              "prev_remove_operator", "prev_insert_operator"]             # categorical columns

# ❶  feature transformer (dense output for easy → torch)
prep = ColumnTransformer(
        transformers=[
            ("pos",    MinMaxScaler(),               num_pos),
            ("signed", MinMaxScaler((-1, 1)),        num_signed),
            ("cat",    OneHotEncoder(sparse_output=False,
                                     handle_unknown="ignore"), cat_cols),
        ])

prep.fit(train_df[num_pos + num_signed + cat_cols])
joblib.dump(prep, "../artifacts/feature_prep.joblib")

['artifacts/feature_prep.joblib']

In [68]:
# prepare scaler for target feature
y_scaler = StandardScaler().fit(
              train_df[["short_long_improvement"]])

joblib.dump(y_scaler, "../artifacts/y_scaler.joblib")

['artifacts/y_scaler.joblib']

## Now split into batches, 6-2-1

In [67]:
# First I will train the NN using only C1
c1_roulette_wheel_training = df.iloc[0:2000*6]
c1_random_training = df.iloc[2000*9:2000*9+2000*6]

In [68]:
c1_roulette_wheel_validation = df.iloc[2000*6:2000*8]
c1_random_validation = df.iloc[2000*9+2000*6:2000*9+2000*8]

In [69]:
c1_roulette_wheel_test = df.iloc[2000*8:2000*9]
c1_random_test = df.iloc[2000*9+2000*8:2000*9+2000*9]

In [37]:
len(c1_roulette_wheel_test)

2000

In [41]:
c1_random_test.head()

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_r_op_4,success_r_op_5,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,short_long_improvement
34000,1,0,1,0,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,-92.845866,9414.022224,3,2,-359.520402
34001,2,0,1,0,5,-92.845866,1.0,0,3.0,2.0,...,1.0,1.0,1.0,1.0,1.0,0.000000,9414.022224,5,2,-294.528296
34002,3,0,1,0,5,-92.845866,1.0,1,5.0,2.0,...,1.0,1.0,1.0,1.0,1.0,-0.754637,9413.267587,5,1,-295.056541
34003,4,0,1,0,5,-0.754637,1.0,0,5.0,1.0,...,1.0,1.0,1.0,1.0,1.0,-3.333590,9409.933997,1,1,-296.861809
34004,5,0,1,0,5,-3.333590,1.0,0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,-340.195001,9069.738996,5,2,-532.664796


In [70]:
c1_training = pd.concat([c1_roulette_wheel_training, c1_random_training], ignore_index=True)
c1_validation = pd.concat([c1_roulette_wheel_validation, c1_random_validation], ignore_index=True)
c1_test = pd.concat([c1_roulette_wheel_test, c1_random_test], ignore_index=True)

In [71]:
c1_training.to_csv('c1_training.csv', header=True, index=False)
c1_validation.to_csv('c1_validation.csv', header=True, index=False)
c1_test.to_csv('c1_test.csv', header=True, index=False)

In [73]:
pd.read_csv('c1_training.csv')

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_r_op_4,success_r_op_5,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.000000,1.00000,1.000000,1.000000,1.000000,-718.948672,9548.083839,1,2,-557.001712
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,-91.810910,9456.272928,1,3,-118.005279
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,-160.354919,9295.918009,4,1,-165.986085
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,9295.918009,5,2,-53.737642
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,9295.918009,3,1,-53.737642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,1996,0,1,0,15,-334.782193,1.0,1,5.0,3.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,-723.808981,12289.819320,1,2,-1011.110096
23996,1997,0,1,0,15,-723.808981,1.0,0,1.0,2.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,-1681.479365,10608.339955,4,2,-1347.567058
23997,1998,0,1,0,15,-1681.479365,1.0,0,4.0,2.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,242.856720,10851.196676,2,3,-0.531799
23998,1999,0,1,0,15,-1681.479365,1.0,1,2.0,3.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,-198.515670,10652.681006,2,3,-309.492472
